In [1]:
import deepcell
import numpy as np
import os
from sklearn.metrics import confusion_matrix, classification_report

## Load data and model

In [2]:
selected_training_data_path = '../sample-data/deepcell/tissue-net/selected-5-train/selected_training_data.npz';
with np.load(selected_training_data_path) as loader:
    selected_training_X = loader['X']
    selected_training_y = loader['y']

In [3]:
from imaging_helpers import make_app

# Assumes that the model has been downloaded already.
model_path = os.path.expanduser('~') + '/.keras/models/MultiplexSegmentation'
app = make_app(model_path=model_path)

2024-03-21 13:35:29.076245: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Loaded model in 29.473865187726915 s


## Generate predictions

Note that we have to select 1:2 in the first axis to keep the axis.

In [4]:
predictions = app.predict(selected_training_X[1:2, ...], image_mpp=0.5)[0]

## Generate classification report

Note that we convert the predictions to binary: the pixel either is, or is not, in a cell.

In [5]:
selected_training_y[selected_training_y > 0] = 1
selected_training_y = selected_training_y.astype(dtype=bool, copy=True)
training_y_1 = selected_training_y[1, :, :, 0].copy()

binary_predictions = predictions.copy()
binary_predictions[binary_predictions > 0] = 1
binary_predictions = binary_predictions.astype(dtype=bool)

print(classification_report(training_y_1.reshape(-1, 1), binary_predictions.reshape(-1, 1)))

              precision    recall  f1-score   support

       False       0.81      0.83      0.82     31733
        True       0.98      0.97      0.97    230411

    accuracy                           0.95    262144
   macro avg       0.89      0.90      0.90    262144
weighted avg       0.96      0.95      0.96    262144

